In [ ]:
# Exemplo de normalização
# Protótipo para treinamento experimental
# 07/03/23

In [ ]:
!pip install pandas==1.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pandas-1.5.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)


In [3]:
# Montagem automática do drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
pd.__version__
dados = pd.read_csv('/content/drive/MyDrive/Sistemas Inteligentes Avançados/dados_normalizar.csv', sep = ';')
#dados.describe()
dados.columns
dados.head()

,idade,altura,Peso,sexo
0,48,1,107,F
1,10,1,240,F
2,79,1,68,F
3,42,1,160,F
4,23,2,157,F


In [7]:
# Normalizar atributos numéricos
dados['novacol']=0 # Criando uma nova coluna no dataframe 

#dados['idade'].min()
#dados['idade'].max()

# Normalizar usando a equação Min/Max 
dados['n_idade'] = (dados['idade']-dados['idade'].min())/(dados['idade'].max()-dados['idade'].min())
dados[['idade', 'n_idade']]

# Normalizar usando a biblioteca
# Passo 1: instalar a lib para normalização
from sklearn import preprocessing #biblioteca com pré processador
# Criando objeto normalizador
normalizador = preprocessing.MinMaxScaler()

# Passo 2: segmentar a base, separando os atributos numéricos
dados_num = dados.drop(columns = ['sexo', 'n_idade', 'novacol'])
dados_num

# Passo 3: obter o modelo normalizador e salvar para uso posterior
# Modelo normalizador possui os pesos!
modelo_normalizador = normalizador.fit(dados_num) # O método fit() é responsável por obter o modelo que normalizará a base
dados_num_normalizado = normalizador.fit_transform(dados_num) # responsável por aplicar a normalização (por isso o termo 'transforma')
# Alternativa ->
#dados_num_normalizado = modelo_normalizador.fit_transform(dados_num) 


# Passo 4: salvar o modelo normalizador em disco 
from pickle import dump, load #compactador
dump(modelo_normalizador, open('normalizador1.model', 'wb'))

dados_num_normalizado = normalizador.fit_transform(dados_num)

dados_num_normalizado

<class 'pandas.core.frame.DataFrame'>


,sexo_F,sexo_M
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
195,0,1
196,0,1
197,1,0
198,0,1


In [8]:
# Normalizar dados categóricos
# Passo 1: Segmentar os dados categóricos
dados_cat = dados[['sexo']] # Com dois colchetes ele converte para Data Frame
print(type(dados_cat))

# Passo 2: normalizar com get_dummies() do pandas
dados_cat_normalizado = pd.get_dummies(dados_cat)
dados_cat_normalizado


<class 'pandas.core.frame.DataFrame'>


,sexo_F,sexo_M
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
195,0,1
196,0,1
197,1,0
198,0,1


In [10]:
# Remontar o Data Frame

# Você pode criar dois objetos separados, um pra normalizar e outro para criar o DataFrame, mas consome bastante memória!
# Ou você pode usar o mesmo, mas isso é code smell

# Reconstruir o Data Frame com os dados numéricos normalizados 
dados_num = pd.DataFrame(dados_num_normalizado, columns = dados_num.columns)
dados_num

# Juntar com os dados categóricos normalizados
dados = dados_num.join(dados_cat_normalizado, how = 'left')
dados

# Verificar se os indexes estão corretos, se não estiverem, regenerar os indexes


,idade,altura,Peso,sexo_F,sexo_M
0,0.474747,0.0,0.432653,1,0
1,0.090909,0.0,0.975510,1,0
2,0.787879,0.0,0.273469,1,0
3,0.414141,0.0,0.648980,1,0
4,0.222222,1.0,0.636735,1,0
...,...,...,...,...,...
195,0.888889,1.0,0.257143,0,1
196,0.141414,0.0,0.195918,0,1
197,0.676768,0.0,0.567347,1,0
198,0.111111,0.0,0.914286,0,1


In [ ]:
# Exemplo para carregar o modelo normalizador a partir do disco
modelo_normalizador = load(open('/content/normalizador1.model', 'rb'))

# Não esquecer de dar close nos arquivos para que o garbage collector limpe da memória